# Loading the Required Libraries and Reading the Data

In [15]:
# !pip install pycaret
#!pip install gradio
import pandas as pd

from pycaret.regression import *

GE = pd.read_csv(r'M:\ISA401-Fa22\GE\Model_Data.csv')

In [3]:
GE.head() 

,esn,dist,flight_cycle,t50_max,nc_avg,ps30_max,nrc_max,bpr_max,w32_avg,rul
0,999101,16086.823174,31,1411.140,9049.054516,47.50,8139.62,8.4577,23.379097,149
1,999102,59807.494851,49,1413.465,9047.997347,47.67,8137.07,8.4992,23.313918,147
2,999103,86928.554029,126,1414.605,9052.272619,47.88,8137.46,8.5043,23.280522,66
3,999104,95777.578284,106,1414.190,9052.378396,47.71,8139.14,8.4910,23.297936,93
4,999105,98745.394623,98,1414.680,9052.542653,47.76,8140.87,8.4885,23.295200,123


## Setting up the Model

In [4]:
setup = setup(data = GE, target ='rul', ignore_features = ['esn'], session_id = 401)

,Description,Value
0,session_id,401
1,Target,rul
2,Original Data,"(100, 10)"
3,Missing Values,False
4,Numeric Features,8
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(69, 8)"


## Training the AdaBoost Regressor Regression Model

In [6]:
ada_model = create_model('ada')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,23.1715,1234.2665,35.1321,0.5401,0.6326,0.6950
1,17.8366,509.7198,22.5770,0.6819,0.2093,0.1713
2,25.0952,728.8313,26.9969,0.8090,0.6142,0.7983
3,23.6582,695.4935,26.3722,0.7037,0.4275,0.4734
4,30.5864,1442.8869,37.9854,0.3941,0.4552,0.4212
5,28.1093,865.0737,29.4121,0.7054,0.7714,1.1725
6,13.2867,275.8004,16.6072,0.8415,0.3280,0.2602
7,16.7606,490.5550,22.1485,0.7315,0.6321,0.8078
8,20.3701,836.0886,28.9152,0.6835,0.5273,0.3335


## Tuning the AdaBoost Model

In [8]:
tuned_ada = tune_model(ada_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,20.9215,951.1906,30.8414,0.6455,0.5647,0.5996
1,17.2837,512.2876,22.6338,0.6803,0.1938,0.1566
2,23.8374,690.7806,26.2827,0.8190,0.6009,0.7678
3,23.5951,746.7109,27.3260,0.6819,0.4076,0.4270
4,33.6997,1623.3085,40.2903,0.3184,0.5245,0.5237
5,28.1281,905.3234,30.0886,0.6917,0.7725,1.1631
6,14.2625,265.1255,16.2827,0.8476,0.2883,0.2436
7,18.7160,590.6027,24.3023,0.6767,0.6210,0.7874
8,17.9758,812.1595,28.4984,0.6926,0.5085,0.2743


## Evaluating Model Performance
The plots below give us insight into how the model is performing, allowing us to analyze its usefulness.

In [9]:
evaluate_model(tuned_ada)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

## Validating Holdout Performance
The output of the model on the holdout data is similar to the validation data, resulting in no indication of overfitting.

In [10]:
pred_holdout = predict_model(tuned_ada)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,24.9381,915.9335,30.2644,0.6668,0.5259,0.5297


## Finalizing the Model
We can now retrain the model over our entire dataset, save it, and then use it in our app to begin predictions.

In [11]:
final_model = finalize_model(tuned_ada)
save_model(final_model, 'M:\\ISA401-Fa22\\GE\\final_model')

In [18]:
temp_df = pd.read_csv(r'M:\ISA401-Fa22\GE\Model_Data.csv')

pred = predict_model(final_model, temp_df)
pred

print({'RUL': pred['Label'][0].astype('float64')}) 

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,14.3642,261.5475,16.1724,0.9055,0.3769,0.3726


{'RUL': 144.25}


In [22]:
import gradio as gr

def predict(dist, flight_cycle, t50_max, nc_avg, ps30_max, nrc_max, bpr_max, w32_avg):
    
    model = load_model('M:\\ISA401-Fa22\\GE\\final_model')
    
    GE_df = pd.DataFrame.from_dict({
     'dist': [dist], 'flight_cycle': [flight_cycle], 't50_max': [t50_max], 'nc_avg': [nc_avg], 'ps30_max': [ps30_max], 
     'nrc_max': [nrc_max], 'bpr_max': [bpr_max], 'w32_avg': [w32_avg]
  })
    pred = predict_model(final_model, temp_df)


    return {'RUL': pred['Label'][0]}

dist = gr.inputs.Number()
flight_cycle = gr.inputs.Number()
t50_max = gr.inputs.Number()
nc_avg = gr.inputs.Number()
ps30_max = gr.inputs.Number()
nrc_max = gr.inputs.Number()
bpr_max = gr.inputs.Number()
w32_avg = gr.inputs.Number()


gr.Interface(predict, [dist, flight_cycle, t50_max, nc_avg, ps30_max, nrc_max, bpr_max, w32_avg], 
            'number',
            live=True,
            title = "Predicting Remaining Useful Life Using AdaBoost Regressor",
            description = "description test",
            css = 'https://bootswatch.com/5/journal/bootstrap.css').launch(debug = False);


Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


Transformation Pipeline and Model Successfully Loaded


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,14.3642,261.5475,16.1724,0.9055,0.3769,0.3726


Traceback (most recent call last):
  File "C:\Users\holroyjm\.conda\envs\GE\lib\site-packages\gradio\routes.py", line 289, in run_predict
    output = await app.blocks.process_api(
  File "C:\Users\holroyjm\.conda\envs\GE\lib\site-packages\gradio\blocks.py", line 983, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "C:\Users\holroyjm\.conda\envs\GE\lib\site-packages\gradio\blocks.py", line 930, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "C:\Users\holroyjm\.conda\envs\GE\lib\site-packages\gradio\components.py", line 544, in postprocess
    return self._round_to_precision(y, self.precision)
  File "C:\Users\holroyjm\.conda\envs\GE\lib\site-packages\gradio\components.py", line 493, in _round_to_precision
    return float(num)
TypeError: float() argument must be a string or a number, not 'dict'
